In [233]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [234]:
base = pd.read_csv('datasets/petr4_treinamento.csv')
base = base.dropna()
base.shape

(1242, 7)

In [235]:
# price Open
base_treinamento = base.iloc[:, 1:2].values

In [236]:
# normalization data
normalizador = MinMaxScaler(feature_range=(0,1))
base_treinamento_normalizada = normalizador.fit_transform(base_treinamento)

In [237]:
base.shape[0]

1242

In [238]:
lenght = base.shape[0]
dsstart = 90

previsores=[]
preco_real=[]

for i in range(dsstart, lenght):
    previsores.append(base_treinamento_normalizada[i-dsstart:i])
    preco_real.append(base_treinamento_normalizada[i])
    
previsores = np.array(previsores)
preco_real = np.array(preco_real)

In [239]:
# organizando a estrutura para o Keras
previsores = np.reshape(previsores, (preco_real.shape[0], previsores.shape[1], 1))

In [240]:
regressor = Sequential()
regressor.add(LSTM(units=100, return_sequences=True, input_shape=(previsores.shape[1], 1)))
regressor.add(Dropout(0.3))# dropar 30% dos dados para evitar overfitting

regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units=50))
regressor.add(Dropout(0.3))

regressor.add(Dense(units=1, activation='linear'))
regressor.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['mean_absolute_error'])

In [241]:
regressor.fit(previsores, preco_real, epochs=100, batch_size=32)

Epoch 1/100
36/36 [==============================] - 3s 89ms/step - loss: 0.0369 - mean_absolute_error: 0.1373
Epoch 2/100
36/36 [==============================] - 3s 93ms/step - loss: 0.0148 - mean_absolute_error: 0.0951: 1s - loss: 0.0141 - mean_a
Epoch 3/100
36/36 [==============================] - 3s 90ms/step - loss: 0.0152 - mean_absolute_error: 0.0969
Epoch 4/100
36/36 [==============================] - 3s 92ms/step - loss: 0.0105 - mean_absolute_error: 0.0792
Epoch 5/100
36/36 [==============================] - 3s 89ms/step - loss: 0.0107 - mean_absolute_error: 0.0804
Epoch 6/100
36/36 [==============================] - 3s 86ms/step - loss: 0.0095 - mean_absolute_error: 0.0738
Epoch 7/100
36/36 [==============================] - 3s 86ms/step - loss: 0.0085 - mean_absolute_error: 0.0696
Epoch 8/100
36/36 [==============================] - 3s 86ms/step - loss: 0.0081 - mean_absolute_error: 0.0682
Epoch 9/100
36/36 [==============================] - 3s 85ms/step - loss: 0.0076 - m

36/36 [==============================] - 3s 90ms/step - loss: 0.0017 - mean_absolute_error: 0.0316: 0s - loss: 0.0018 - mean_absolute_erro
Epoch 73/100
36/36 [==============================] - 3s 90ms/step - loss: 0.0018 - mean_absolute_error: 0.0320
Epoch 74/100
36/36 [==============================] - 3s 90ms/step - loss: 0.0019 - mean_absolute_error: 0.0317
Epoch 75/100
36/36 [==============================] - 3s 90ms/step - loss: 0.0018 - mean_absolute_error: 0.0320
Epoch 76/100
36/36 [==============================] - 3s 93ms/step - loss: 0.0017 - mean_absolute_error: 0.0313
Epoch 77/100
36/36 [==============================] - 3s 87ms/step - loss: 0.0017 - mean_absolute_error: 0.0312: 1s - loss: 0.0017 - mean_absolu
Epoch 78/100
36/36 [==============================] - 3s 84ms/step - loss: 0.0017 - mean_absolute_error: 0.0313
Epoch 79/100
36/36 [==============================] - 3s 90ms/step - loss: 0.0016 - mean_absolute_error: 0.0298
Epoch 80/100
36/36 [========================

# Predict

In [242]:
base_teste = pd.read_csv('datasets/petr4_teste.csv')
preco_real_teste = base_teste.iloc[:, 1:2].values
base_teste.shape

(22, 7)

In [243]:
base_completa = pd.concat((base['Open'], base_teste['Open']), axis = 0)
base_completa.shape

(1264,)

In [244]:
entradas = base_completa[len(base_completa) - len(base_teste) - 90:].values
entradas = entradas.reshape(-1, 1)
entradas = normalizador.transform(entradas)
entradas.shape

(112, 1)

In [245]:
X_teste=[]
for i in range(dsstart, entradas.shape[0]):
    X_teste.append(entradas[i-dsstart:i])
X_teste = np.array(X_teste)
X_teste = np.reshape(X_teste, (X_teste.shape[0], X_teste.shape[1], 1))
X_teste.shape

(22, 90, 1)

In [246]:
previsoes = regressor.predict(X_teste)

In [247]:
previsoes = normalizador.inverse_transform(previsoes)

In [248]:
len(preco_real_teste), len(previsoes)

(22, 22)

In [257]:
previsoes.mean(), preco_real_teste.mean(), (abs(previsoes.mean()-preco_real_teste.mean()))

(17.908348, 17.87454563636364, 0.03380244713245517)

In [258]:
plt.figure(figsize=(15,6))
preco_real_teste.plot(color='red', label='Preço Real')
plt.plot(previsoes, color='blue', label='Previsões')
plt.grid(True)
plt.legend()

AttributeError: 'numpy.ndarray' object has no attribute 'plot'

<Figure size 1080x432 with 0 Axes>